# 聚类：K-均值算法

### 1 概述

聚类是一种无监督的学习，它将相似的对象归到同一个簇中。

“类”指的是具有相似性的集合。聚类是指将数据集划分为若干类，使得类内之间的数据最为相似，各类之间的数据相似度差别尽可能大。聚类分析就是以相似性为基础，对数据集进行聚类划分，属于无监督学习。

* 优点：容易实现。
* 缺点：可能收敛到局部最小值，在大规模数据集上收敛较慢。
* 适用数据类型：数值型数据。

### 2 K-均值聚类算法

K-均值是发现给定数据集的k个簇的算法。簇个数k是用户给定的，每一个簇通过其质心
（centroid），即簇中所有点的中心来描述。

首先，随机确定k个初始点作为质心。然后将数据集中的每个点分配到一个簇中，具体来讲，为每个点找距其最近的质心，并将其分配给该质心所对应的簇。这一步完成之后，每个簇的质心更新为该簇所有点的平均值。

```
创建k个点作为起始质心（经常是随机选择）
当任意一个点的簇分配结果发生改变时
    对数据集中的每个数据点
        对每个质心
            计算质心与数据点之间的距离
        将数据点分配到距其最近的簇
    对每一个簇，计算簇中所有点的均值并将均值作为质心
```

1. 收集数据：使用任意方法。
2. 准备数据：需要数值型数据来计算距离，也可以将标称型数据映射为二值型数据再用于距离计算。
3. 分析数据：使用任意方法。
4. 训练算法：不适用于无监督学习，即无监督学习没有训练过程。
5. 测试算法：应用聚类算法、观察结果。可以使用量化的误差指标如误差平方和（后面会介绍）来评价算法的结果。
6. 使用算法：可以用于所希望的任何应用。通常情况下，簇质心可以代表整个簇的数据来做出决策。

In [1]:
from numpy import *

def loadDataSet(fileName):      #general function to parse tab -delimited floats
    dataMat = []                #assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine)) #map all elements to float()
        dataMat.append(fltLine)
    return dataMat

def distEclud(vecA, vecB): # 计算欧式距离
    return sqrt(sum(power(vecA - vecB, 2))) #la.norm(vecA-vecB)

def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k,n)))#create centroid mat
    for j in range(n): #create random cluster centers, within bounds of each dimension
        minJ = min(dataSet[:,j]) 
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = mat(minJ + rangeJ * random.rand(k,1))
    return centroids

In [2]:
def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))#create mat to assign data points 
                                      #to a centroid, also holds SE of each point
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        print(centroids)
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A==cent)[0]]#get all the point in this cluster
            centroids[cent,:] = mean(ptsInClust, axis=0) #assign centroid to mean 
    return centroids, clusterAssment

In [3]:
dataMat = mat(loadDataSet('../data/kMeansTestSet.txt'))

In [4]:
randCent(dataMat, 2)

matrix([[ 1.21006843, -1.10251447],
        [-4.2940001 , -2.73661984]])

In [5]:
distEclud(dataMat[0], dataMat[1])

5.184632816681332

In [6]:
myCentroids, clustAssing = kMeans(dataMat, 4)

[[ 0.63948606 -4.09426653]
 [-1.14711269  3.93359659]
 [-0.0790007   4.21058793]
 [ 3.18579463  2.00196445]]
[[-0.72175297 -3.03774939]
 [-2.46154315  2.78737555]
 [ 1.57467675  4.0701135 ]
 [ 3.0313103   2.08375545]]
[[-0.83188333 -2.97988206]
 [-2.46154315  2.78737555]
 [ 2.2429152   3.9007646 ]
 [ 3.461461    1.20462093]]
[[-0.92973646 -3.00632531]
 [-2.46154315  2.78737555]
 [ 2.28430469  3.66626685]
 [ 3.64888508  0.51304183]]
[[-1.43308103 -3.01692045]
 [-2.46154315  2.78737555]
 [ 2.51964406  3.40459212]
 [ 3.54444742 -1.31307892]]
[[-3.01169468 -3.01238673]
 [-2.46154315  2.78737555]
 [ 2.54391447  3.21299611]
 [ 3.09814284 -2.43041226]]
[[-3.38237045 -2.9473363 ]
 [-2.46154315  2.78737555]
 [ 2.6265299   3.10868015]
 [ 2.80293085 -2.7315146 ]]


### 3 后处理提高聚类性能

K-均值算法收敛但聚类效果较差的原因是，K-均值算法收敛到了局部最小值，而非全局最小值（局部最小值指结果还可以但并非最好结果，全局最小值是可能的最好结果）。

一种用于度量聚类效果的指标是SSE（Sum of Squared Error，误差平方和），SSE值越小表示数据点越接近于它们的质心，聚类效果也越好。

可以对生成的簇进行后处理，一种方法是将具有最大SSE值的簇划分成两个簇。具体实现时可以将最大簇包含的点过滤出来并在这些点上运行K-均值算法，其中的k设为2。

为了保持簇总数不变，可以将某两个簇进行合并。从图10-2中很明显就可以看出，应该将图
下部两个出错的簇质心进行合并。可以很容易对二维数据上的聚类进行可视化，但是如果遇到40维的数据应该如何去做？

有两种可以量化的办法：合并最近的质心，或者合并两个使得SSE增幅最小的质心。第一种
思路通过计算所有质心之间的距离，然后合并距离最近的两个点来实现。第二种方法需要合并两
个簇然后计算总SSE值。必须在所有可能的两个簇上重复上述处理过程，直到找到合并最佳的两
个簇为止。

### 4 二分K-均值算法

```
将所有点看成一个簇
当簇数目小于k时
    对于每一个簇
        计算总误差
        在给定的簇上面进行K-均值聚类（k=2）
        计算将该簇一分为二之后的总误差
    选择使得误差最小的那个簇进行划分操作
```

In [7]:
def biKmeans(dataSet, k, distMeas=distEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet, axis=0).tolist()[0]
    centList =[centroid0] #create a list with one centroid
    for j in range(m):#calc initial Error
        clusterAssment[j,1] = distMeas(mat(centroid0), dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A==i)[0],:]#get the data points currently in cluster i
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)
            sseSplit = sum(splitClustAss[:,1])#compare the SSE to the currrent minimum
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A!=i)[0],1])
            print("sseSplit, and notSplit: ",sseSplit,sseNotSplit)
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList) #change 1 to 3,4, or whatever
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        print('the bestCentToSplit is: ',bestCentToSplit)
        print('the len of bestClustAss is: ', len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]#replace a centroid with two best centroids 
        centList.append(bestNewCents[1,:].tolist()[0])
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:]= bestClustAss#reassign new clusters, and SSE
    return mat(centList), clusterAssment

In [8]:
centList, myNewAssments = biKmeans(dataMat, 3)

[[ 3.59462014  2.75825073]
 [ 0.86787263 -3.13161489]]
[[ 0.56274151  2.86795713]
 [-0.73746283 -2.62210322]]
[[ 0.18713124  2.8560699 ]
 [-0.40926764 -2.89114795]]
[[ 0.08249337  2.94802785]
 [-0.2897198  -2.83942545]]
sseSplit, and notSplit:  792.9168565373268 0.0
the bestCentToSplit is:  0
the len of bestClustAss is:  80
[[4.20415483 1.77725177]
 [0.29158938 3.5557368 ]]
[[ 3.2843811   2.6674442 ]
 [-0.98480253  3.04155573]]
[[ 2.71358074  3.11839563]
 [-2.29801424  2.79388557]]
[[ 2.6265299   3.10868015]
 [-2.46154315  2.78737555]]
sseSplit, and notSplit:  66.36683512000786 466.63278133614426
[[-2.76304144 -0.78119154]
 [-2.01204929 -1.56346232]]
[[-4.3014205  -2.47945167]
 [ 0.41822738 -2.90295024]]
[[-3.53973889 -2.89384326]
 [ 2.65077367 -2.79019029]]
sseSplit, and notSplit:  84.25921395268443 326.2840752011824
the bestCentToSplit is:  1
the len of bestClustAss is:  40


In [9]:
centList

matrix([[ 0.08249337,  2.94802785],
        [-3.53973889, -2.89384326],
        [ 2.65077367, -2.79019029]])